FinBuddy Step 1: Extract and Prepare 18K Dataset

In [ ]:
# Install required libraries
!pip install datasets pandas

import pandas as pd
from datasets import load_dataset
import json


# 1. Load the 18K Dataset from Hugging Face


In [ ]:
print("Loading dataset from Hugging Face...")
ds = load_dataset("Akhil-Theerthala/Kuvera-PersonalFinance-V2.1")

# Convert to pandas DataFrame for easier manipulation
df = pd.DataFrame(ds['train'])

print(f"Dataset loaded! Total rows: {len(df)}")
print(f"Columns: {df.columns.tolist()}")
print("\nFirst row preview:")
print(df.head(1))

Loading dataset from Hugging Face...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

final.jsonl:   0%|          | 0.00/116M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/18846 [00:00<?, ? examples/s]

Dataset loaded! Total rows: 18846
Columns: ['category', 'query', 'chain_of_thought', 'response']

First row preview:
                   category  \
0  Debt Management & Credit   

                                               query  \
0  "Help! My friend's ex-wife might have stolen h...   

                                    chain_of_thought  \
0  Phase 1: Query Analysis\nThe user's core dilem...   

                                            response  
0  I hear how overwhelming this must be for both ...  



# 2. Explore Category Distribution


In [ ]:
print("\n" + "="*60)
print("CATEGORY DISTRIBUTION")
print("="*60)
category_counts = df['category'].value_counts()
print(category_counts)


CATEGORY DISTRIBUTION
category
Debt Management & Credit            5175
Retirement Planning                 3286
Tax Planning & Optimization         3019
Investing & Wealth Building         2994
Budgeting & Cash Flow Management    2503
Insurance & Risk Management         1035
Savings & Emergency Funds            638
Estate Planning & Legacy             196
Name: count, dtype: int64



# 3. Clean and Structure the Data


In [ ]:
print("\n" + "="*60)
print("CLEANING DATA")
print("="*60)

# Add unique query IDs
df['query_id'] = ['Q' + str(i).zfill(5) for i in range(len(df))]

# Select and rename columns for clarity
queries_df = df[['query_id', 'category', 'query', 'chain_of_thought', 'response']].copy()

# Remove any null values
queries_df = queries_df.dropna()

print(f"Cleaned dataset: {len(queries_df)} queries")


CLEANING DATA
Cleaned dataset: 18846 queries



# 4. Sample Queries for Synthetic Generation


In [ ]:
# For 120 user profiles, we'll use ~40-50 diverse queries
# Each query will get 2-3 user profiles with different financial contexts

print("\n" + "="*60)
print("SAMPLING QUERIES FOR SYNTHETIC DATA")
print("="*60)

# Sample 50 queries (stratified by category)
sample_size = 50
sampled_queries = queries_df.groupby('category', group_keys=False).apply(
    lambda x: x.sample(min(len(x), max(1, int(sample_size * len(x) / len(queries_df)))))
).reset_index(drop=True)

print(f"Sampled {len(sampled_queries)} queries across categories:")
print(sampled_queries['category'].value_counts())


SAMPLING QUERIES FOR SYNTHETIC DATA
Sampled 46 queries across categories:
category
Debt Management & Credit            13
Retirement Planning                  8
Tax Planning & Optimization          8
Investing & Wealth Building          7
Budgeting & Cash Flow Management     6
Insurance & Risk Management          2
Estate Planning & Legacy             1
Savings & Emergency Funds            1
Name: count, dtype: int64


/tmp/ipython-input-1745488286.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_queries = queries_df.groupby('category', group_keys=False).apply(


# 5. Save to CSV

In [ ]:
print("\n" + "="*60)
print("SAVING FILES")
print("="*60)

# Save full dataset
queries_df.to_csv('queries_dataset_full.csv', index=False)
print("Saved: queries_dataset_full.csv")

# Save sampled queries (for synthetic generation)
sampled_queries.to_csv('queries_dataset_sampled.csv', index=False)
print("Saved: queries_dataset_sampled.csv")


SAVING FILES
Saved: queries_dataset_full.csv
Saved: queries_dataset_sampled.csv




# 6. Create Category Analysis for Synthetic Personas


In [ ]:
category_analysis = {
    'category': [],
    'count': [],
    'common_themes': []
}

for category in sampled_queries['category'].unique():
    cat_queries = sampled_queries[sampled_queries['category'] == category]
    category_analysis['category'].append(category)
    category_analysis['count'].append(len(cat_queries))

    # Extract first 100 chars of each query as themes
    themes = cat_queries['query'].str[:100].tolist()[:3]
    category_analysis['common_themes'].append(' | '.join(themes))

category_df = pd.DataFrame(category_analysis)
category_df.to_csv('category_analysis.csv', index=False)
print("Saved: category_analysis.csv")

Saved: category_analysis.csv




# 7. Preview Sample Query


In [ ]:
print("\n" + "="*60)
print("SAMPLE QUERY PREVIEW")
print("="*60)

sample = sampled_queries.iloc[0]
print(f"Query ID: {sample['query_id']}")
print(f"Category: {sample['category']}")
print(f"\nQuery: {sample['query'][:200]}...")
print(f"\nResponse: {sample['response'][:200]}...")

print("\n" + "="*60)
print("STEP 1 COMPLETE! ✅")
print("="*60)



SAMPLE QUERY PREVIEW
Query ID: Q08138
Category: Budgeting & Cash Flow Management

Query: Okay, I'm trying to buy my first home in Dallas and I'm kinda freaking out about whether this is even possible. The houses I'm looking at are around $250-300K, but I've only got about $10K to put down...

Response: Based on your situation and concerns, here's a realistic path forward:

First, take a breath – your anxiety is completely understandable, but homeownership *is* achievable for you. Your strong credit ...

STEP 1 COMPLETE! ✅



#FinBuddy Step 2: Generate Synthetic User Profiles & Transactions



In [ ]:
#This creates 120 realistic user profiles with transaction histories
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
import os

# Set seed for reproducibility
np.random.seed(42)
random.seed(42)


# 1. Load Sampled Queries

In [ ]:
print("Loading sampled queries...")
queries_df = pd.read_csv('queries_dataset_sampled.csv')
print(f"Loaded {len(queries_df)} queries")

Loading sampled queries...
Loaded 46 queries



# 2. Define User Personas (Archetypes)


In [ ]:

PERSONAS = {
    'young_professional': {
        'age_range': (25, 35),
        'income_range': (45000, 85000),
        'spending_patterns': {
            'Dining': (200, 500),
            'Entertainment': (100, 300),
            'Shopping': (150, 400),
            'Groceries': (300, 500),
            'Rent': (1200, 2000),
            'Utilities': (100, 200),
            'Transportation': (100, 300)
        },
        'debt_likelihood': 0.3,
        'savings_rate': 0.15
    },
    'student': {
        'age_range': (18, 25),
        'income_range': (15000, 35000),
        'spending_patterns': {
            'Dining': (150, 300),
            'Entertainment': (50, 150),
            'Groceries': (200, 350),
            'Rent': (600, 1000),
            'Education': (500, 2000),
            'Transportation': (50, 150)
        },
        'debt_likelihood': 0.7,  # Student loans
        'savings_rate': 0.05
    },
    'family_household': {
        'age_range': (35, 55),
        'income_range': (65000, 120000),
        'spending_patterns': {
            'Groceries': (600, 1000),
            'Dining': (300, 600),
            'Childcare': (500, 1500),
            'Utilities': (200, 400),
            'Mortgage': (1500, 3000),
            'Insurance': (300, 600),
            'Healthcare': (200, 500)
        },
        'debt_likelihood': 0.6,  # Mortgage, auto
        'savings_rate': 0.10
    },
    'retiree': {
        'age_range': (60, 80),
        'income_range': (30000, 60000),
        'spending_patterns': {
            'Groceries': (300, 500),
            'Healthcare': (400, 800),
            'Utilities': (150, 300),
            'Dining': (150, 300),
            'Entertainment': (100, 250)
        },
        'debt_likelihood': 0.2,
        'savings_rate': -0.05  # Drawing down savings
    },
    'debt_heavy': {
        'age_range': (25, 55),
        'income_range': (35000, 65000),
        'spending_patterns': {
            'Credit Card Payment': (200, 800),
            'Loan Payment': (300, 1000),
            'Groceries': (250, 450),
            'Dining': (100, 250),
            'Rent': (800, 1400),
            'Utilities': (100, 200)
        },
        'debt_likelihood': 0.95,
        'savings_rate': -0.02
    },
    'high_earner': {
        'age_range': (35, 60),
        'income_range': (120000, 250000),
        'spending_patterns': {
            'Investment': (2000, 5000),
            'Dining': (500, 1200),
            'Shopping': (400, 1000),
            'Travel': (500, 2000),
            'Mortgage': (3000, 6000),
            'Groceries': (600, 1000)
        },
        'debt_likelihood': 0.4,
        'savings_rate': 0.25
    }
}


# 3. Map Categories to Personas

In [ ]:
CATEGORY_PERSONA_MAP = {
    'Debt Management & Credit': ['debt_heavy', 'student', 'young_professional'],
    'Investment & Savings': ['young_professional', 'high_earner', 'family_household'],
    'Budgeting & Expense': ['student', 'young_professional', 'family_household'],
    'Retirement Planning': ['retiree', 'high_earner', 'family_household'],
    'Insurance': ['family_household', 'young_professional', 'retiree'],
    'Tax': ['high_earner', 'family_household', 'young_professional'],
    'Emergency Fund': ['young_professional', 'family_household', 'student']
}


# 4. Generate User Profiles

In [ ]:
print("\n" + "="*60)
print("GENERATING USER PROFILES")
print("="*60)

user_profiles = []
user_id = 1

# For each query, create 2-3 user profiles with relevant personas
for _, query_row in queries_df.iterrows():
    query_id = query_row['query_id']
    category = query_row['category']

    # Get relevant personas for this category
    relevant_personas = CATEGORY_PERSONA_MAP.get(category, ['young_professional'])
    num_users = random.randint(2, 3)

    for _ in range(num_users):
        persona_type = random.choice(relevant_personas)
        persona = PERSONAS[persona_type]

        age = random.randint(*persona['age_range'])
        income = random.randint(*persona['income_range'])

        # Create financial situation description based on query
        financial_situation = query_row['query'][:100]

        user_profiles.append({
            'user_id': f'U{str(user_id).zfill(3)}',
            'query_id': query_id,
            'persona_type': persona_type,
            'age': age,
            'income_monthly': income // 12,
            'income_annual': income,
            'financial_situation': financial_situation,
            'has_debt': random.random() < persona['debt_likelihood']
        })

        user_id += 1

        if user_id > 120:
            break

    if user_id > 120:
        break

user_profiles_df = pd.DataFrame(user_profiles)
print(f"Generated {len(user_profiles_df)} user profiles")
print(f"\nPersona distribution:")
print(user_profiles_df['persona_type'].value_counts())


GENERATING USER PROFILES
Generated 109 user profiles

Persona distribution:
persona_type
young_professional    70
debt_heavy            12
student                8
family_household       8
retiree                6
high_earner            5
Name: count, dtype: int64


# 5. Generate Transaction Histories

In [ ]:
print("\n" + "="*60)
print("GENERATING TRANSACTION HISTORIES")
print("="*60)

# Create transactions folder
os.makedirs('transactions', exist_ok=True)

def generate_transactions(user_profile, num_months=4):
    """Generate realistic transaction history for a user"""
    persona = PERSONAS[user_profile['persona_type']]
    transactions = []

    start_date = datetime.now() - timedelta(days=num_months*30)
    current_balance = random.randint(500, 5000)

    # Generate monthly income
    for month in range(num_months):
        # Salary deposit
        income_date = start_date + timedelta(days=month*30 + random.randint(1, 5))
        income_amount = user_profile['income_monthly'] * random.uniform(0.95, 1.05)
        current_balance += income_amount

        transactions.append({
            'date': income_date.strftime('%Y-%m-%d'),
            'amount': round(income_amount, 2),
            'category': 'Income',
            'merchant': 'Employer Salary Deposit',
            'description': 'Monthly salary',
            'balance': round(current_balance, 2)
        })

        # Generate expenses throughout the month
        for _ in range(random.randint(15, 30)):
            days_offset = random.randint(0, 28)
            trans_date = income_date + timedelta(days=days_offset)

            # Pick random spending category
            category = random.choice(list(persona['spending_patterns'].keys()))
            amount_range = persona['spending_patterns'][category]
            amount = -random.uniform(*amount_range)

            # Generate merchant name
            merchants = {
                'Groceries': ['Walmart', 'Kroger', 'Whole Foods', 'Target'],
                'Dining': ['Chipotle', 'Starbucks', 'Local Restaurant', 'McDonald\'s'],
                'Entertainment': ['Netflix', 'Spotify', 'Movie Theater', 'Amazon'],
                'Shopping': ['Amazon', 'Target', 'Macy\'s', 'Best Buy'],
                'Rent': ['Property Management Co', 'Landlord'],
                'Mortgage': ['Bank Mortgage Payment'],
                'Utilities': ['Electric Company', 'Water & Sewer', 'Internet Provider'],
                'Transportation': ['Gas Station', 'Uber', 'Metro Transit'],
                'Credit Card Payment': ['Credit Card Payment'],
                'Loan Payment': ['Student Loan', 'Auto Loan', 'Personal Loan'],
                'Healthcare': ['Pharmacy', 'Doctor Office', 'Health Insurance'],
                'Insurance': ['Auto Insurance', 'Home Insurance'],
                'Education': ['University Tuition', 'Textbooks'],
                'Childcare': ['Daycare Center', 'After School Program'],
                'Investment': ['Brokerage Transfer', '401k Contribution'],
                'Travel': ['Airline', 'Hotel', 'Airbnb']
            }

            merchant = random.choice(merchants.get(category, ['Misc Merchant']))

            current_balance += amount

            transactions.append({
                'date': trans_date.strftime('%Y-%m-%d'),
                'amount': round(amount, 2),
                'category': category,
                'merchant': merchant,
                'description': f'{category} expense',
                'balance': round(current_balance, 2)
            })

    # Sort by date
    transactions_df = pd.DataFrame(transactions)
    transactions_df = transactions_df.sort_values('date').reset_index(drop=True)

    return transactions_df

# Generate transactions for each user
for idx, user in user_profiles_df.iterrows():
    transactions = generate_transactions(user)
    filename = f"transactions/user_{user['user_id']}_transactions.csv"
    transactions.to_csv(filename, index=False)

    if (idx + 1) % 20 == 0:
        print(f"Generated {idx + 1}/{len(user_profiles_df)} transaction files...")

print(f"✅ Generated {len(user_profiles_df)} transaction CSV files")



GENERATING TRANSACTION HISTORIES
Generated 20/109 transaction files...
Generated 40/109 transaction files...
Generated 60/109 transaction files...
Generated 80/109 transaction files...
Generated 100/109 transaction files...
✅ Generated 109 transaction CSV files


# 6. Save User Profiles

In [ ]:
user_profiles_df.to_csv('user_profiles.csv', index=False)
print("✅ Saved: user_profiles.csv")

✅ Saved: user_profiles.csv




# 7. Preview Sample Data


In [ ]:
print("\n" + "="*60)
print("SAMPLE USER PROFILE & TRANSACTIONS")
print("="*60)

sample_user = user_profiles_df.iloc[0]
print(f"\nUser ID: {sample_user['user_id']}")
print(f"Persona: {sample_user['persona_type']}")
print(f"Age: {sample_user['age']}")
print(f"Monthly Income: ${sample_user['income_monthly']:,.2f}")
print(f"Has Debt: {sample_user['has_debt']}")
print(f"Financial Situation: {sample_user['financial_situation']}")

sample_trans = pd.read_csv(f"transactions/user_{sample_user['user_id']}_transactions.csv")
print(f"\nTransaction Count: {len(sample_trans)}")
print("\nFirst 5 transactions:")
print(sample_trans.head())

print("\nSpending by category:")
spending = sample_trans[sample_trans['amount'] < 0].groupby('category')['amount'].sum().sort_values()
print(spending)

print("\n" + "="*60)
print("STEP 2 COMPLETE! ✅")
print("="*60)
print("\nGenerated files:")
print("- user_profiles.csv")
print("- transactions/ folder with 120 CSV files")



SAMPLE USER PROFILE & TRANSACTIONS

User ID: U001
Persona: young_professional
Age: 29
Monthly Income: $5,087.00
Has Debt: True
Financial Situation: Okay, I'm trying to buy my first home in Dallas and I'm kinda freaking out about whether this is eve

Transaction Count: 92

First 5 transactions:
         date   amount       category                 merchant  \
0  2025-07-27  4907.61         Income  Employer Salary Deposit   
1  2025-07-28 -1399.36           Rent   Property Management Co   
2  2025-07-31  -146.25  Entertainment                  Spotify   
3  2025-08-03  -233.96       Shopping                   Amazon   
4  2025-08-03  -259.97       Shopping                   Macy's   

             description  balance  
0         Monthly salary  7247.61  
1           Rent expense  3963.30  
2  Entertainment expense   416.41  
3       Shopping expense   562.66  
4       Shopping expense   796.62  

Spending by category:
category
Rent             -23697.24
Groceries         -5560.39
Enter


# FinBuddy Step 3: Prepare Training Data & LoRA Fine-tuning

# PART A: INSTALL AND IMPORT

In [ ]:
!pip install -q transformers peft accelerate bitsandbytes datasets

import pandas as pd
import json
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    BitsAndBytesConfig,
    Trainer,
    DataCollatorForLanguageModeling
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import Dataset
import os

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 45.1 MB/s eta 0:00:00
PyTorch version: 2.9.0+cu126
CUDA available: True


# PART B: CREATE TRAINING EXAMPLES (HYBRID)

In [ ]:
print("\n" + "="*60)
print("CREATING TRAINING EXAMPLES (HYBRID APPROACH)")
print("="*60)

# Load data
queries_df = pd.read_csv('queries_dataset_full.csv')  # Use FULL dataset
user_profiles_df = pd.read_csv('user_profiles.csv')

print(f"Full dataset: {len(queries_df)} queries")
print(f"User profiles: {len(user_profiles_df)} synthetic users")

def format_transactions_for_prompt(transactions_df, max_rows=20):
    """Convert transaction CSV to readable text format"""
    recent = transactions_df.tail(max_rows)

    total_income = recent[recent['amount'] > 0]['amount'].sum()
    total_expenses = abs(recent[recent['amount'] < 0]['amount'].sum())
    current_balance = recent.iloc[-1]['balance'] if 'balance' in recent.columns else 0

    spending_by_cat = recent[recent['amount'] < 0].groupby('category')['amount'].sum()
    top_categories = spending_by_cat.sort_values().head(3)

    summary = f"""Financial Summary:
- Current Balance: ${current_balance:,.2f}
- Income (last period): ${total_income:,.2f}
- Expenses (last period): ${total_expenses:,.2f}
- Top spending: {', '.join([f'{cat} (${abs(amt):,.0f})' for cat, amt in top_categories.items()])}

Recent Transactions:
"""

    for _, row in recent.iterrows():
        summary += f"- {row['date']}: {row['category']} at {row['merchant']} (${row['amount']:,.2f})\n"

    return summary


CREATING TRAINING EXAMPLES (HYBRID APPROACH)
Full dataset: 18846 queries
User profiles: 109 synthetic users


# 1. GENERIC EXAMPLES (from full 18K dataset)

In [ ]:
print("\n1. Creating generic training examples from full dataset...")
generic_training = []

for idx, row in queries_df.iterrows():
    instruction = f"""You are FinBuddy, a personal finance advisor. Answer the following question with helpful financial advice.

User Question: {row['query']}

Provide clear, actionable financial advice."""

    generic_training.append({
        'instruction': instruction,
        'response': row['response']
    })

    if (idx + 1) % 2000 == 0:
        print(f"  Processed {idx + 1}/{len(queries_df)} queries...")

print(f"✅ Created {len(generic_training)} generic examples")


1. Creating generic training examples from full dataset...
  Processed 2000/18846 queries...
  Processed 4000/18846 queries...
  Processed 6000/18846 queries...
  Processed 8000/18846 queries...
  Processed 10000/18846 queries...
  Processed 12000/18846 queries...
  Processed 14000/18846 queries...
  Processed 16000/18846 queries...
  Processed 18000/18846 queries...
✅ Created 18846 generic examples


# 2. PERSONALIZED EXAMPLES (from synthetic data)

In [ ]:
print("\n2. Creating personalized examples with transaction context...")
personalized_training = []

for _, user in user_profiles_df.iterrows():
    query_row = queries_df[queries_df['query_id'] == user['query_id']].iloc[0]

    # Load user transactions
    trans_file = f"transactions/user_{user['user_id']}_transactions.csv"
    transactions = pd.read_csv(trans_file)

    # Format transactions
    trans_summary = format_transactions_for_prompt(transactions)

    # Create instruction with context
    instruction = f"""You are FinBuddy, a personal finance advisor. A user has shared their transaction history and asked a question. Provide personalized financial advice based on their specific situation.

{trans_summary}

User Question: {query_row['query']}

Analyze their financial situation and provide specific, actionable advice."""

    # Personalize response
    income = transactions[transactions['amount'] > 0]['amount'].sum() / 4
    expenses = abs(transactions[transactions['amount'] < 0]['amount'].sum()) / 4
    balance = transactions.iloc[-1]['balance'] if 'balance' in transactions.columns else 0

    personalized_intro = f"""Based on your transaction history, I can see you have a monthly income of approximately ${income:,.0f} and monthly expenses around ${expenses:,.0f}, with a current balance of ${balance:,.0f}. """

    full_response = personalized_intro + query_row['response']

    personalized_training.append({
        'instruction': instruction,
        'response': full_response
    })

print(f"✅ Created {len(personalized_training)} personalized examples")


2. Creating personalized examples with transaction context...
✅ Created 109 personalized examples


# 3. COMBINE DATASETS

In [ ]:
print("\n3. Combining datasets...")

# Combine all training data
all_training_data = generic_training + personalized_training

# Shuffle to mix generic and personalized
import random
random.shuffle(all_training_data)

print(f"\n📊 Training Data Summary:")
print(f"  Generic examples: {len(generic_training)}")
print(f"  Personalized examples: {len(personalized_training)}")
print(f"  Total examples: {len(all_training_data)}")

# Sample some personalized examples more (to emphasize personalization)
print("\n4. Oversampling personalized examples (3x) for emphasis...")
personalized_repeated = personalized_training * 3
all_training_data_weighted = generic_training + personalized_repeated
random.shuffle(all_training_data_weighted)

print(f"  Weighted total: {len(all_training_data_weighted)}")



3. Combining datasets...

📊 Training Data Summary:
  Generic examples: 18846
  Personalized examples: 109
  Total examples: 18955

4. Oversampling personalized examples (3x) for emphasis...
  Weighted total: 19173


# PART C: FORMAT FOR QWEN

In [ ]:
print("\n" + "="*60)
print("FORMATTING FOR QWEN MODEL")
print("="*60)

def format_for_qwen(example):
    """Format in Qwen's chat template"""
    return f"""<|im_start|>system
You are FinBuddy, an expert personal finance advisor. You analyze transaction histories and provide personalized financial advice.<|im_end|>
<|im_start|>user
{example['instruction']}<|im_end|>
<|im_start|>assistant
{example['response']}<|im_end|>"""

# Use the weighted dataset (with oversampled personalized examples)
formatted_data = [{"text": format_for_qwen(ex)} for ex in all_training_data_weighted]

# Split data (90/10)
train_size = int(0.9 * len(formatted_data))
train_data = formatted_data[:train_size]
eval_data = formatted_data[train_size:]

print(f"Training examples: {len(train_data)}")
print(f"Evaluation examples: {len(eval_data)}")

# Save to JSONL
with open('training_data.jsonl', 'w') as f:
    for item in train_data:
        f.write(json.dumps(item) + '\n')

with open('eval_data.jsonl', 'w') as f:
    for item in eval_data:
        f.write(json.dumps(item) + '\n')

print("✅ Saved training_data.jsonl and eval_data.jsonl")

# Preview both types
print("\n" + "="*60)
print("SAMPLE GENERIC EXAMPLE:")
print("="*60)
generic_sample = [ex for ex in all_training_data_weighted if 'transaction' not in ex['instruction'].lower()][0]
print(format_for_qwen(generic_sample)[:400] + "...")

print("\n" + "="*60)
print("SAMPLE PERSONALIZED EXAMPLE:")
print("="*60)
personalized_sample = [ex for ex in all_training_data_weighted if 'transaction' in ex['instruction'].lower()][0]
print(format_for_qwen(personalized_sample)[:400] + "...")



FORMATTING FOR QWEN MODEL
Training examples: 17255
Evaluation examples: 1918
✅ Saved training_data.jsonl and eval_data.jsonl

SAMPLE GENERIC EXAMPLE:
<|im_start|>system
You are FinBuddy, an expert personal finance advisor. You analyze transaction histories and provide personalized financial advice.<|im_end|>
<|im_start|>user
You are FinBuddy, a personal finance advisor. Answer the following question with helpful financial advice.

User Question: I finally crawled out of debt and want to make sure I'm not missing anything with my money now. At 2...

SAMPLE PERSONALIZED EXAMPLE:
<|im_start|>system
You are FinBuddy, an expert personal finance advisor. You analyze transaction histories and provide personalized financial advice.<|im_end|>
<|im_start|>user
You are FinBuddy, a personal finance advisor. Answer the following question with helpful financial advice.

User Question: *"Hey, I’m really confused and kinda stressed about something with my 401k. At the end of November, ...


# PART D: LOAD MODEL WITH QUANTIZATION

In [ ]:
print("\n" + "="*60)
print("LOADING QWEN MODEL")
print("="*60)

model_name = "Qwen/Qwen2.5-1.5B-Instruct"

# Quantization config for 4-bit (QLoRA)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Load model with quantization
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

model = prepare_model_for_kbit_training(model)

print("✅ Model loaded with 4-bit quantization")



LOADING QWEN MODEL


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

✅ Model loaded with 4-bit quantization


# PART E: CONFIGURE LORA

In [ ]:
print("\n" + "="*60)
print("CONFIGURING LORA")
print("="*60)

lora_config = LoraConfig(
    r=16,                       # LoRA rank
    lora_alpha=32,              # Scaling factor
    target_modules=[            # Qwen attention layers
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj"
    ],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


CONFIGURING LORA
trainable params: 18,464,768 || all params: 1,562,179,072 || trainable%: 1.1820


# PART F: PREPARE DATASET

In [ ]:
def load_jsonl(file_path):
    data = []
    with open(file_path, 'r') as f:
        for line in f:
            data.append(json.loads(line))
    return data

train_dataset = Dataset.from_list(load_jsonl('training_data.jsonl'))
eval_dataset = Dataset.from_list(load_jsonl('eval_data.jsonl'))

print(f"Training samples: {len(train_dataset)}")
print(f"Evaluation samples: {len(eval_dataset)}")

Training samples: 17255
Evaluation samples: 1918


# PART G: TOKENIZE DATASETS FIRST

In [ ]:
from transformers import Trainer, DataCollatorForLanguageModeling

print("\n" + "="*60)
print("TOKENIZING DATASETS")
print("="*60)

# Tokenization function
def tokenize_function(examples):
    """Tokenize the text data"""
    result = tokenizer(
        examples["text"],
        truncation=True,
        max_length=2048,
    )
    return result

# Tokenize datasets
tokenized_train = train_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=["text"],
    desc="Tokenizing training data"
)

tokenized_eval = eval_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=["text"],
    desc="Tokenizing eval data"
)

print(f"Training samples after tokenization: {len(tokenized_train)}")
print(f"Evaluation samples after tokenization: {len(tokenized_eval)}")

# Custom data collator
class DataCollatorForCompletionOnlyLM(DataCollatorForLanguageModeling):
    def __call__(self, features):
        batch = {
            "input_ids": [f["input_ids"] for f in features],
            "attention_mask": [f["attention_mask"] for f in features]
        }

        batch = self.tokenizer.pad(
            batch,
            padding=True,
            max_length=2048,
            return_tensors="pt"
        )

        batch["labels"] = batch["input_ids"].clone()
        batch["labels"][batch["labels"] == self.tokenizer.pad_token_id] = -100

        return batch

data_collator = DataCollatorForCompletionOnlyLM(tokenizer=tokenizer, mlm=False)



TOKENIZING DATASETS


Tokenizing training data:   0%|          | 0/17255 [00:00<?, ? examples/s]

Tokenizing eval data:   0%|          | 0/1918 [00:00<?, ? examples/s]

Training samples after tokenization: 17255
Evaluation samples after tokenization: 1918


PART H: TRAINING CONFIGURATION (NOW AFTER TOKENIZATION)

In [ ]:
print("\n" + "="*60)
print("TRAINING SETUP")
print("="*60)

training_args = TrainingArguments(
    output_dir="./finbuddy_lora",
    num_train_epochs=2,
    per_device_train_batch_size=8,           # For A100
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=50,
    logging_first_step=True,
    eval_strategy="steps",
    eval_steps=500,
    save_strategy="steps",
    save_steps=1000,
    save_total_limit=3,
    warmup_steps=100,
    weight_decay=0.01,
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    report_to="none",
    load_best_model_at_end=True,
    metric_for_best_model="loss",
    greater_is_better=False,
    disable_tqdm=False,
    gradient_checkpointing=False,
    dataloader_num_workers=2,
)

total_steps = len(tokenized_train) // (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps) * training_args.num_train_epochs

print(f"Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"Total training samples: {len(tokenized_train)}")
print(f"Estimated total steps: ~{total_steps}")
print(f"Estimated training time on A100: ~{total_steps * 4 / 60:.1f} minutes")



TRAINING SETUP
Effective batch size: 16
Total training samples: 17255
Estimated total steps: ~2156
Estimated training time on A100: ~143.7 minutes


# PART I: CREATE TRAINER

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    data_collator=data_collator,
)

print("✅ Trainer created successfully!")

✅ Trainer created successfully!


# PART J: START TRAINING

In [ ]:
print("\n" + "="*60)
print("STARTING TRAINING")
print("="*60)

try:
    train_result = trainer.train()

    print("\n✅ Training complete!")
    print(f"Final training loss: {train_result.training_loss:.4f}")
    print(f"Training steps completed: {train_result.global_step}")

except Exception as e:
    print(f"❌ Training error: {str(e)}")
    raise


STARTING TRAINING


You're using a Qwen2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:2779: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
You're using a Qwen2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:2779: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
`use_cache=True` is incompatible with gradient c

You're using a Qwen2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a Qwen2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:2779: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:2779: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
You're using a Qwen2TokenizerFast tokenizer. Ple


✅ Training complete!
Final training loss: 1.5917
Training steps completed: 2158


# SAVE MODEL

In [ ]:
print("\n" + "="*60)
print("SAVING MODEL")
print("="*60)

# Check training happened
if train_result.global_step == 0:
    print("⚠️ WARNING: No training steps completed!")
else:
    print(f"✅ Trained for {train_result.global_step} steps")

# Save LoRA adapters
model.save_pretrained("finbuddy_lora_final")
tokenizer.save_pretrained("finbuddy_lora_final")

print("✅ Model saved to: finbuddy_lora_final/")

# List saved files
import os
saved_files = os.listdir("finbuddy_lora_final")
print(f"Saved files ({len(saved_files)}): {saved_files[:5]}...")


SAVING MODEL
✅ Trained for 2158 steps
✅ Model saved to: finbuddy_lora_final/
Saved files (10): ['added_tokens.json', 'adapter_config.json', 'vocab.json', 'merges.txt', 'chat_template.jinja']...


Test Inference

In [ ]:
print("\n" + "="*60)
print("TESTING INFERENCE")
print("="*60)

model.eval()

test_prompt = """<|im_start|>system
You are FinBuddy, an expert personal finance advisor.<|im_end|>
<|im_start|>user
I have $5000 in credit card debt at 18% APR. My monthly income is $3500 and expenses are $2800. Should I pay off debt or save for emergency fund first?<|im_end|>
<|im_start|>assistant
"""

print("Generating response...")
inputs = tokenizer(test_prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=300,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.1
    )

response = tokenizer.decode(outputs[0], skip_special_tokens=False)

try:
    assistant_response = response.split("<|im_start|>assistant")[-1].split("<|im_end|>")[0].strip()
    print("\n📝 Assistant Response:")
    print(assistant_response[:500])  # First 500 chars
except:
    print("\n⚠️ Response parsing issue, showing raw:")
    print(response[:500])

print("\n✅ Inference test complete!")


TESTING INFERENCE
Generating response...

📝 Assistant Response:
Based on your situation, here's a clear breakdown of the two paths you're considering:

**Option 1: Pay Off the Debt Immediately (Recommended Path):**
This means using your entire $5000 credit card balance to pay it off today. While this guarantees a guaranteed 18% "return" by avoiding future interest costs, there's a significant risk – if your expenses rise unexpectedly or you face unexpected income changes, this could leave you with little cash flow left each month. This path requires strict b

✅ Inference test complete!


In [ ]:
import shutil
from google.colab import files

# Zip the model folder
print("Creating zip file...")
shutil.make_archive('finbuddy_lora_final', 'zip', 'finbuddy_lora_final')

# Download
print("Starting download...")
files.download('finbuddy_lora_final.zip')

print("✅ Model downloaded! (~100-200 MB)")

Creating zip file...
Starting download...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Model downloaded! (~100-200 MB)
